## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization

"""
建立神經網路，並加入 BN layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 0.001
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [7]:
# 載入 Callbacks
"""Code Here
設定 callbacks: model checkpoint
"""
from keras.callbacks import ModelCheckpoint

model_ckpt_T = ModelCheckpoint(filepath="./tmp_T.h5", 
                             monitor="val_loss", 
                             save_best_only=True)

model_ckpt_F = ModelCheckpoint(filepath="./tmp_F.h5", 
                             monitor="val_loss", 
                             save_best_only=False)

model_ckpt_W = ModelCheckpoint(filepath="./best_weight.h5", 
                             monitor="val_loss", 
                             save_best_only=True,
                             save_weights_only=True)


results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
mtd = ['Only_Weight','Best_False','Best_True']

for i,ckpt in enumerate([model_ckpt_W,model_ckpt_F,model_ckpt_T]):

    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[ckpt]
            )

    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["accuracy"]
    valid_acc = model.history.history["val_accuracy"]

    name_tag = f"ckpt_{mtd[i]}"
    results[name_tag] = {'train-loss': train_loss,
                        'valid-loss': valid_loss,
                        'train-acc': train_acc,
                        'valid-acc': valid_acc}


#這邊就存最後一次跑的 (best_only = True)
model.save("final_model.h5")
model.save_weights("model_weights.h5")
pred_final = model.predict(x_test)

# Load back
model_save = keras.models.load_model("./tmp_T.h5")
pred_loadback = model_save.predict(x_test)
loss_loadback, acc_loadback = model_save.evaluate(x_test, y_test)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)              

Epoch 49/50
49/49 [==============================] - 3s 57ms/step - loss: 0.5111 - accuracy: 0.8563 - val_loss: 1.6974 - val_accuracy: 0.4852
Epoch 50/50
49/49 [==============================] - 3s 58ms/step - loss: 0.4958 - accuracy: 0.8627 - val_loss: 1.7039 - val_accuracy: 0.4841
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_3 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_4 (Batch (None

Epoch 47/50
49/49 [==============================] - 3s 59ms/step - loss: 0.5375 - accuracy: 0.8444 - val_loss: 1.6493 - val_accuracy: 0.4936
Epoch 48/50
49/49 [==============================] - 3s 61ms/step - loss: 0.5199 - accuracy: 0.8522 - val_loss: 1.6649 - val_accuracy: 0.4816
Epoch 49/50
49/49 [==============================] - 3s 62ms/step - loss: 0.5047 - accuracy: 0.8585 - val_loss: 1.6596 - val_accuracy: 0.4902
Epoch 50/50
49/49 [==============================] - 3s 64ms/step - loss: 0.4889 - accuracy: 0.8637 - val_loss: 1.6996 - val_accuracy: 0.4888
Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normaliz

Epoch 45/50
49/49 [==============================] - 3s 70ms/step - loss: 0.5888 - accuracy: 0.8247 - val_loss: 1.5951 - val_accuracy: 0.4876
Epoch 46/50
49/49 [==============================] - 3s 67ms/step - loss: 0.5711 - accuracy: 0.8311 - val_loss: 1.6074 - val_accuracy: 0.4936
Epoch 47/50
49/49 [==============================] - 3s 65ms/step - loss: 0.5544 - accuracy: 0.8367 - val_loss: 1.6258 - val_accuracy: 0.4876
Epoch 48/50
49/49 [==============================] - 3s 63ms/step - loss: 0.5384 - accuracy: 0.8425 - val_loss: 1.6204 - val_accuracy: 0.4928
Epoch 49/50
49/49 [==============================] - 3s 61ms/step - loss: 0.5217 - accuracy: 0.8507 - val_loss: 1.6738 - val_accuracy: 0.4867
Epoch 50/50
313/313 [==============================] - 1s 2ms/step - loss: 1.4448 - accuracy: 0.4983


In [8]:
from sklearn.metrics import accuracy_score

final_model_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=pred_final.argmax(axis=-1))
loadback_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=pred_loadback.argmax(axis=-1))

print("Accuracy of final weights: %.3f" % final_model_acc)
print("Accuracy of best weights: %.3f" % loadback_acc)

Accuracy of final weights: 0.485
Accuracy of best weights: 0.498


# 果然儲存的最佳模型較高

In [9]:
new_model = build_mlp(input_shape=x_train.shape[1:])
new_model_pred = new_model.predict(x_test)
new_model_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=new_model_pred.argmax(axis=-1))
print("Accuracy of new model: %.3f" % new_model_acc)

new_model.load_weights("./model_weights.h5")
new_model_pred = new_model.predict(x_test)
new_model_loadback_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=new_model_pred.argmax(axis=-1))
print("Accuracy of final weights: %.3f" % new_model_loadback_acc)

new_model.load_weights("./best_weight.h5")
new_model_pred = new_model.predict(x_test)
new_model_loadback_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=new_model_pred.argmax(axis=-1))
print("Accuracy of best weights: %.3f" % new_model_loadback_acc)

Accuracy of new model: 0.098
Accuracy of final weights: 0.485
Accuracy of best weights: 0.488


In [10]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""

'Code Here\n將結果繪出\n'